<a href="https://colab.research.google.com/github/benjarojas/ADA-Informes/blob/main/OptimalBST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Optimal Binary Search Tree

**Entrada:** Un arreglo $K$ **ordenado** de $n$ claves $K = [k_1, k_2, \ldots , k_n]$. Cada clave $k_i$ tiene asociada una probabilidad $p_i$ de ser buscada, por lo que en la entrada también tenemos un arreglo $P$ de probabilidades, que quedaría como $P = [p_1, p_2, \ldots, p_n]$

**Salida:** Un **árbol de búsqueda** cuya distribución de nodos **minimice la cantidad media de nodos** que debe visitar el algoritmo de búsqueda para encontrar una clave.

Un **árbol binario de búsqueda óptimo** es **árbol de búsqueda** donde el coste medio de buscar un elemento se reduce al mínimo, con esto logramos que incluso con **aproximaciones de los costos de búsqueda se aceleran considerablemente las búsquedas medias.**

# 2. Código

## 2.1 OptimalBST Recursivo

In [43]:
from termcolor import colored

def optimalCost(prob, i, j, verbose=False): 
  global call_count

  # caso base
  if j < i:     
    return 0
  if j == i:
    if(verbose): print(f"Caso base (j == i). Retornando prob[{i}] = {colored(f'{prob[i]}', 'blue')}\n") 
    return prob[i]

  # sumamos todas las probabilidades del sub-arreglo
  fsum = auxSum(prob, i, j)
  if(verbose): print(f"Suma de las probabilidades entre p[{i}] y p[{j}]: {fsum}")
  
  minCost = float('inf') # inicializamos en infinito para la primera comparación
  for r in range(i, j + 1): # iteramos por cada índice del arreglo, r será la raíz del árbol
    call_count = call_count + 1
    A = optimalCost(prob, i, r - 1, verbose=verbose)
    if(verbose): print(f"Calculamos recursivamente el coste óptimo entre {colored(f'p[{i}]', 'red')} y {colored(f'p[{r-1}]', 'red')}: {colored(f'{A}', 'green')}")
      
    call_count = call_count + 1
    B = optimalCost(prob, r + 1, j, verbose=verbose)
    if(verbose): print(f"Calculamos recursivamente el coste óptimo entre {colored(f'p[{r+1}]', 'red')} y {colored(f'p[{j}]', 'red')}: {colored(f'{A}', 'green')}")

    cost = A + B
    if(verbose): print(f"Costo calculado: {cost}\n")

    if cost < minCost:
      if(verbose): print(f"El costo calculado {cost} es menor que el costo mínimo anterior {minCost}\n")
      minCost = cost
     
  if(verbose): print(f"Costo óptimo: {minCost + fsum}\n")
  return minCost + fsum

def recursiveOptBST(arrayKeys, prob, n, verbose=False):
  return optimalCost(prob, 0, n - 1, verbose=verbose)
 
# función auxiliar para sumar todas las probabilidades
# entre los índices i y j de un arreglo
def auxSum(prob, i, j):
  s = 0
  for k in range(i, j + 1):
    s += prob[k]
  return s

In [28]:
k=[1, 2, 3, 4, 5]
p=[0.35, 0.2, 0.15, 0.2, 0.1]
call_count = 0
n = len(k)
print(f"Costo mínimo esperado: {recursiveOptBST(k,p,n)}")
print(f"Llamadas recursivas: {call_count}")

Costo mínimo esperado: 2.05
Llamadas recursivas: 134


## 2.2 OptimalBST Bottom-Up (Programación Dinámica)

In [45]:
import numpy as np
from termcolor import colored

def optimalBST(arrayKeys, arrayProb,verbose=False):
  global subCount
  cantKeys = len(arrayKeys)  #cantidad de llaves

  if(verbose):
    print(f"{str(colored('Keys:', 'red'))} {arrayKeys}")
    print(f"{str(colored('Probabilidades:', 'blue'))} {arrayProb}\n")

  costMatrix = np.zeros((cantKeys+2,cantKeys+1)) 
  probMatrix = np.zeros((cantKeys+2,cantKeys+1))

  # Iteramos la matriz por sobre la diagonal principal
  for i in range(0,cantKeys+1):
    for j in range(1, cantKeys+1-i): 
      probMatrix[j][i+j] = probMatrix[j][i+j-1] + arrayProb[i+j-1] #se calcula la probabilidad en la posición evaluada

      if(verbose):
        print(f"{str(colored(f'Probabilidad para probMatrix[{j}][{i+j}]: {probMatrix[j][i+j]}', 'blue'))}")

      minCost = float('inf')
      for r in range (j,i+j+1): # iteramos por todas las raíces posibles de esa posición
        subCount += 1
        cost = probMatrix[j][i+j] + costMatrix[j][r-1] + costMatrix[r+1][i+j] # calculamos el costo

        if(verbose):
          print(f"Costo para {str(colored(f'r={r}','red'))}: {str(colored(f'{cost}', 'green'))}")

        if(cost<minCost): #se busca el mínimo costo entre las raíces trabajadas
          minCost = cost

      costMatrix[j][i+j] = minCost
      if(verbose == True):
          print(f"{str(colored('Menor costo de las raices calculadas de', 'green'))} {str(colored(f'probMatrix[{j}][{i+j}]', 'blue'))}{str(colored(f': {minCost}', 'green'))}\n")
  
  if(verbose):
    print(f"{str(colored('Matriz de probabilidades:', 'blue'))} \n{probMatrix}\n")
    print(f"{str(colored('Matriz de costos:', 'green'))} \n{costMatrix}\n")
    print(f"{str(colored(f'Costo mínimo esperado: {costMatrix[1][cantKeys]}', 'green'))}")
  
  return(costMatrix[1][cantKeys])

In [27]:
k=[1, 2, 3, 4, 5]
p=[0.35, 0.2, 0.15, 0.2, 0.1]
subCount = 0
print(f"Costo mínimo esperado: {optimalBST(k,p)}")
print(f"Cantidad de subproblemas: {subCount}")

Costo mínimo esperado: 2.05
Cantidad de subproblemas: 35


## 2.3 Generador de Instancias

In [30]:
import random

def optimal_bst_instance_generator(n):
    keys = sorted(random.sample(range(1, 100), n))
    arr = np.random.random(n*2+1)
    arr /= arr.sum()
    p = list(arr[:n]) # Probabilidad de las claves
    return keys, p

# 3. Descripción de los algoritmos

## 3.1 OptimalBST Recursivo

### 3.1.1 Descripción

Recibimos como entrada un arreglo de probabilidades y un par de índices $i, j$ que corresponden al **sub-árbol cuyo costo óptimo se calcula en la llamada recursiva**.

El algoritmo ejecuta las siguientes operaciones:



1.   Se calcula la suma de todas las probabilidades en el árbol $prob[i \ldots j]$
2.   Iterativamente tomamos cada nodo $i$ como raíz $r$ del árbol
3.   Recursivamente calculamos el coste óptimo para el **sub-árbol izquierdo** y **derecho** a partir de la raíz $r$.
4.   El **costo esperado** se calcula como el **costo del sub-arbol izquierdo** + **costo del sub-arbol derecho** + **la suma de las probabilidades**.
5.   Por cada iteración se compara el **costo mínimo** con el **costo mínimo anterior** y se guarda el que sea menor.
6.   Se reconstruye el árbol a partir de la raíz y los sub-arboles que **minimicen el costo**.
7.   Retornam el **costo mínimo esperado**.

### 3.1.2 Ejemplo de ejecución

### 3.1.3 Ejecución paso a paso (`verbose=True`)

In [41]:
keys, prob = optimal_bst_instance_generator(5)
recursiveOptBST(keys, prob, len(prob), verbose=True)

Suma de las probabilidades entre p[0] y p[4]: 0.4918520002654707
Calculamos recursivamente el coste óptimo entre p[0] y p[-1]: 0
Suma de las probabilidades entre p[1] y p[4]: 0.46425833735304106
Calculamos recursivamente el coste óptimo entre p[1] y p[0]: 0
Suma de las probabilidades entre p[2] y p[4]: 0.4065081411183751
Calculamos recursivamente el coste óptimo entre p[2] y p[1]: 0
Suma de las probabilidades entre p[3] y p[4]: 0.23323369630239169
Calculamos recursivamente el coste óptimo entre p[3] y p[2]: 0
Caso base (j == i). Retornando prob[4] = 0.10225912007285631

Calculamos recursivamente el coste óptimo entre p[4] y p[4]: 0
Costo calculado: 0.10225912007285631

El costo calculado 0.10225912007285631 es menor que el costo mínimo anterior inf

Caso base (j == i). Retornando prob[3] = 0.13097457622953537

Calculamos recursivamente el coste óptimo entre p[3] y p[3]: 0.13097457622953537
Calculamos recursivamente el coste óptimo entre p[5] y p[4]: 0.13097457622953537
Costo calculado:

0.940282338700244

## 3.2 OptimalBST Bottom-Up (Programación Dinámica)

### 3.2.1 Descripción

Recibimos como entrada un **arreglo de claves** y un **arreglo de probabilidades** *(cada clave tiene asociada una probabilidad)*.


1.   Primero el algoritmo crea **2 matrices de dimensión $(n+1) \times (n+2)$** donde $n$ es la **cantidad de keys** del árbol.
2.   Las matrices corresponden a **costMatrix** y **probMatrix**, es decir, las matrices de costo y de probabilidad.
3.   Se calcula el valor de $\text{costMatrix}[i][j]$ y de $\text{probMatrix}[i][j]$ desde la diagonal principal hacia arriba.
4.   $\text{costMatrix}[i][j] = {probMatrix}[i][j] + {costMatrix}[i][r-1] + {costMatrix}[r+1][j]$ donde $r$ itera desde $i$ hasta $j$.
5.   ${probMatrix}[i][j] = {probMatrix}[i][j-1] + {prob}[j-1]$
6.   Retornamos el valor contenido en {costMatrix}[1][n], que corresponde al costo esperado del árbol óptimo.



### 3.2.2 Ejemplo de ejecución

### 3.2.3 Ejecución paso a paso (`verbose=True`)

In [46]:
keys, prob = optimal_bst_instance_generator(5)
optimalBST(keys, prob, verbose=True)

Keys: [13, 14, 55, 66, 76]
Probabilidades: [0.054152098743014535, 0.08190626300374555, 0.0945626102098999, 0.12746478345713977, 0.11952812278470369]

Probabilidad para probMatrix[1][1]: 0.054152098743014535
Costo para r=1: 0.054152098743014535
Menor costo de las raices calculadas de probMatrix[1][1]: 0.054152098743014535

Probabilidad para probMatrix[2][2]: 0.08190626300374555
Costo para r=2: 0.08190626300374555
Menor costo de las raices calculadas de probMatrix[2][2]: 0.08190626300374555

Probabilidad para probMatrix[3][3]: 0.0945626102098999
Costo para r=3: 0.0945626102098999
Menor costo de las raices calculadas de probMatrix[3][3]: 0.0945626102098999

Probabilidad para probMatrix[4][4]: 0.12746478345713977
Costo para r=4: 0.12746478345713977
Menor costo de las raices calculadas de probMatrix[4][4]: 0.12746478345713977

Probabilidad para probMatrix[5][5]: 0.11952812278470369
Costo para r=5: 0.11952812278470369
Menor costo de las raices calculadas de probMatrix[5][5]: 0.11952812278470

0.9764776818927815